Copyright (C) 2024 Pablo Castells y Alejandro Bellogín

El código que contiene este notebook se ha implementado para la realización de las prácticas de la asignatura "Sistemas de recomendación" del Máster en Ciencia de Datos, impartido en la Escuela Politécnica Superior de la Universidad Autónoma de Madrid. El fin del mismo, así como su uso, se ciñe a las actividades docentes de dicha asignatura.

### **Sistemas de recomendación 2024-25**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Máster en Ciencia de Datos

# Filtrado colaborativo con aprendizaje profundo: EASE, Two-Tower, Transformers (EN CONSTRUCCIÓN)

Fechas:

* Comienzo: martes 25 de febrero.
* Entrega: lunes 17 de marzo, 23:59.

## Objetivos

Esta práctica tiene por objetivo comprender el diseño de métodos de filtrado colaborativo mediante deep learning como transición desde un modelo bilineal típico de factorización de matrices hacia modelos neuronales de complejidad arbitraria. En este bloque se desarrollarán:

* Algoritmos de filtrado colaborativo basados en aprendizaje profundo.
* Algoritmos de filtrado colaborativo orientados a datos secuenciales.
* Métricas de evaluación de sistemas de recomendación.

## Material proporcionado

Al igual que en la P1, se proporcionan software y datos para la realización de la práctica:

* Algunas estructuras de datos ya implementadas, para manejar datos de ratings y la salida de los recomendadores.
* Un esqueleto de clases y funciones donde el estudiante desarrollará sus implementaciones. 
  - Se proporciona una celda de prueba al final de este notebook que deberá funcionar con las implementaciones del estudiante.
  - Junto a la celda de prueba en este mismo notebook, se muestra como referencia un ejemplo de salida generada con una implementación de los profesores.
* Los mismos conjuntos de datos de ratings que se usaban en la P1:
  - Dos conjuntos de juguete para prueba y depuración: <ins>toy1.csv</ins> y <ins>toy2.csv</ins> con ratings ficticios.
  - Un conjunto de datos reales de ratings a películas: *ml-1m.zip* disponible en la Web de [MovieLens](https://grouplens.org/datasets/movielens/1m). De los archivos disponibles, se utilizará sólamente <ins>ratings.dat</ins>, añadiéndole una cabecera `u::i::r::t`.
  
Los esqueletos de código que se proporcionan aquí son a modo de guía: el estudiante puede modificarlo todo libremente, siempre que la celda de prueba funcione correctamente **sin cambios**.

En concreto, si para la P1 el estudiante ya hubiera hecho cambios en alguna de estas clases, puede continuar usando dichas modificaciones.

La entrega consistirá en un fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

La celda de prueba deberá ejecutar sin errores a la primera con el código entregado por el estudiante (naturalmente con salvedad de los ejercicios que no se hayan implementado).

## Estructuras de datos: ratings y recomendaciones

Se proporcionan:
* Una clase Ratings que permite leer los datos de un fichero de texto, así como un método que genera dos particiones (de forma <b>aleatoria</b> o <b>temporal</b>) de entrenamiento y test, para evaluar y comparar la efectividad de diferentes algoritmos de recomendación.
* Se pueden reutilizar las clases Recommender y Recommendation de la práctica anterior.

In [ ]:
import numpy as np
import pandas as pd

class Ratings:
    def __init__(self, file=None, sep=','):
        if file:
            data = pd.read_csv(file, delimiter=sep, engine='python')
            u, i, r, t = data.columns[0:4]
            data.r = 1
            self.m = data.pivot(index=u, columns=i, values=r).fillna(0).to_numpy(dtype=np.float32)
            self.mt = data.pivot(index=u, columns=i, values=t).fillna(-1).to_numpy(dtype=np.float32)
            self.uids = np.sort(data[u].unique())
            self.iids = np.sort(data[i].unique())
            self.uidxs = {u:j for j, u in enumerate(self.uids)}
            self.iidxs = {i:j for j, i in enumerate(self.iids)}
            self._nratings = (self.m > 0).sum()
            self.data = data
        
    def copy(self, ratings, matrix, temp_matrix):
        self.m = matrix
        self.mt = temp_matrix
        self.uids = ratings.uids
        self.iids = ratings.iids
        self.uidxs = ratings.uidxs
        self.iidxs = ratings.iidxs
        self._nratings = (matrix > 0).sum()
        dfr = pd.DataFrame(columns=self.iids, index=self.uids, data=self.m).unstack().reset_index(name='r')
        dfr.columns = ['i', 'u', 'r']
        dft = pd.DataFrame(columns=self.iids, index=self.uids, data=self.mt).unstack().reset_index(name='t')
        dft.columns = ['i', 'u', 't']
        df_key = ['u','i']
        df = pd.concat([dfr.set_index(df_key).squeeze(), dft.set_index(df_key).squeeze()], keys = ['r','t'],axis=1).fillna(0).reset_index()
        self.data = df[df.r>0][['u', 'i', 'r', 't']].sort_values(by=['u', 'i'])
        return self
    
    def matrix(self):
        return self.m

    def temporal_matrix(self):
        return self.mt

    def nusers(self):
        return len(self.uids)
    
    def nitems(self):
        return len(self.iids)
    
    # uidx can be an int or an array-like of ints.
    def uidx_to_uid(self, uidx):
        return self.uids[uidx]
        
    # iidx can be an int or an array-like of ints.
    def iidx_to_iid(self, iidx):
        return self.iids[iidx]
    
    def uid_to_uidx(self, uid):
        return self.uidxs[uid]
        
    def iid_to_iidx(self, iid):
        return self.iidxs[iid]
        
    def iidx_rated_by(self, uidx):
        self.m[uidx].nonzero()
        
    def uidx_who_rated(self, iidx):
        self.m[:, iidx].nonzero()
        
    def random_split(self, ratio):
        mask = np.random.choice([True, False], size=self.m.shape, p=[ratio, 1-ratio])
        train = self.m * mask
        temp_train = self.mt * mask
        test = self.m * ~mask
        temp_test = self.mt * ~mask
        return Ratings().copy(self, train, temp_train), Ratings().copy(self, test, temp_test)
    
    def peruser_sequence_split(self, ntestitems=1):
        test_ids_arr = [group.sort_values(by='t', ascending=False)[['u', 'i']].to_numpy() 
                    for _, group in self.data.groupby(by='u')]
        test_ids = []
        for user_arr in test_ids_arr:
            for ids in user_arr[:ntestitems]:
                test_ids.append(ids)
        #print(test_ids)
        test_idx = np.array([[self.uid_to_uidx(uid), self.iid_to_iidx(iid)] for uid, iid in test_ids])
        mask = np.ones(self.matrix().shape)
        mask[test_idx[:, 0], test_idx[:, 1]] = 0
        train = self.m * mask
        temp_train = self.mt * mask
        test = self.m * (1-mask)
        temp_test = self.mt * (1-mask)
        return Ratings().copy(self, train, temp_train), Ratings().copy(self, test, temp_test)
    
    #
    # The remaining functions are just for debugging purposes.
    #

    def rating(self, uid, iid):
        return self.matrix()[self.uid_to_uidx(uid), self.iid_to_iidx(iid)]

    def items_rated_by(self, uid):
        return self.iidx_to_iid(self.iidx_rated_by(self.uid_to_uidx(uid)))
        
    def users_who_rated(self, iid):
        return self.uidx_to_uid(self.uidx_who_rated(self.iid_to_iidx(iid)))
    
    def user_ratings(self, uid):
        iidxs = self.matrix()[self.uid_to_uidx(uid)].nonzero()[0]
        return {self.iidx_to_iid(iidx): fround(r) for iidx, r in zip(iidxs, self.matrix()[self.uid_to_uidx(uid), iidxs])}

    def item_ratings(self, iid):
        uidxs = self.matrix()[:, self.iid_to_iidx(iid)].nonzero()[0]
        return {self.uidx_to_uid(uidx): fround(r) for uidx, r in zip(uidxs, self.matrix()[uidxs, self.iid_to_iidx(iid)])}

    def nratings(self):
        return self._nratings
 

## Ejercicio 1: EASE

Implementar un modelo de filtrado colaborativo lineal basado en autoencoders.

In [ ]:
class Ease(Recommender):
    def __init__(self, training, l=20, threshold=3):
        super().__init__(training)
        # Your code here...

        self.scores = # Your code here...


### Ejercicio 1 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 2: Factorización de matrices: modelo deep learning

Como alternativa a la implementación realizada en la P1 del modelo de factorización de matrices, en esta práctica vas a reformular esa implementación como un caso particular "degenerado" de arquitectura neuronal.

### Implementación en TensorFlow

Completar los huecos marcados con `# Your code here...`.

Observaciones:
* Por la estructura de datos de entrenamiento que maneja TensorFlow, entrenar con toda la matriz de ratings (incluyendo todas las celdas sin dato) es demasiado costoso. Por ello se tomará una muestra pequeña de ejemplos negativos en cada época.
* En el esqueleto que aquí se proporciona, no se genera la traza (curva) de P@10 durante el entrenamiento ya que no encaja fácilmente en el API Keras de TensorFlow.

In [ ]:
import tensorflow as tf
from tqdm.keras import TqdmCallback
import time, datetime

class DLMFRecommender(Recommender):
    def __init__(self, training, k=50, lrate=0.01, nepochs=150, neg=4):
        super().__init__(training)
        # Create the model - this will directly trigger training.        
        tf.random.set_seed(0) # For comparability and debugging (the randomness here is in parameter initialization).
        self.model, self.hist = self.create_model(training, k, lrate, neg, nepochs)
        # Plot the training error and report the final test metric value (P@10).
        # Your code here...

        uexplode = np.full((training.nitems(), training.nusers()), np.arange(training.nusers())).T.flatten()
        iexplode = np.full((training.nusers(), training.nitems()), np.arange(training.nitems())).flatten()
        self.scores = self.model.predict([uexplode, iexplode], batch_size=training.nusers()*100, 
                            verbose=1).reshape(training.nusers(), training.nitems())

    def create_model(self, ratings, k, lrate, neg, nepochs):
        # 'users' is an input layer of type tf.int64.
        users = # Your code here...
        user_embeddings = # Your code here...
        # 'items' is an input layer of type tf.int64.
        items = # Your code here...
        item_embeddings = # Your code here...
        # TensorFlow has a built-in dot-product layer.
        dot = # Your code here...
        # Now we need a generic model that wraps up the "network", specifying the input and output layers.
        model = # Your code here...

        # Compile the model: Adam optimizer is suggested here over SGD.
        # Your code here...

        # Show the model topology
        model.summary()
        tf.keras.utils.plot_model(model, show_shapes=True, dpi=150)
        
        hist = self.train_model(ratings, model, neg, nepochs)
        return model, hist

    def train_model(self, ratings, tf_mf, neg, nepochs):
        # We inject 'neg' negative samples for every available rating in the training data
        nneg = neg * ratings.nratings()
        user_ids = np.concatenate(([ratings.uid_to_uidx(u) for u in ratings.data.u], np.random.choice(list(ratings.uidxs.values()), size=nneg)))
        item_ids = np.concatenate(([ratings.iid_to_iidx(u) for u in ratings.data.i], np.random.choice(list(ratings.iidxs.values()), size=nneg)))
        rs = ratings.matrix()[user_ids, item_ids]
        batch_size = ratings.nratings() + nneg # Single batch with all the data at once.
        
        # Your code here... to actually do the training
        hist = # Your code here...
            , callbacks=[TqdmCallback(verbose=0)]) # Produces a prettier progress bar.
        return hist

### Ejercicio 2 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 3: Implementación modelo Two-Tower

Implementar tu propia versión de un modelo Two-Tower a partir de la arquitectura implementada de MF en el ejercicio anterior. 

In [ ]:
class TwoTowerRecommender(DLMFRecommender):
    # Your code here... Puede no ser necesario re-implementar todos los métodos...

### Ejercicio 3 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 4: Recomendación secuencial [PENDIENTE]

...

### Ejercicio 4 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 5: Ampliaciones

Explorar variaciones sobre una o varias de las implementaciones anteriores, tales como:

* En general:
    * Además de las métricas de evaluación usadas en la P1 (precision, recall) incluir otras métricas, bien de acierto (NDCG) u otras (cobertura, diversidad, etc.)

* Sobre el ejercicio 1 (algoritmo EASE):
    * Añadir la opción de asignar 0 a los pesos negativos, comprobando que su eficacia disminuye (como indica el artículo original)

* Sobre el ejercicio 2 (factorización de matrices por aprendizaje profundo) y 3 (modelo Two-Tower):
    * Diferentes funciones de scoring pérdida: sigmoide / BCE loss, BCE loss with logits.
    * Diferentes optimizadores y configuraciones de los mismos (SGD, Adam, etc.).
    * Variaciones en los hiperparámetros y configuración del modelo: learning rate, número de factores k, número de épocas, inicialización de parámetros del modelo, etc.
    * Añadir opciones tales como regularización, dropout, etc.
    * Añadir capas ocultas en la implementación sobre framework de deep learning.
    * Explorar una formulación *pairwise learning to rank* sobre MF (p.e., BPR).

* Sobre el ejercicio 4 (recomendación secuencial):
    * ...
    * ...
    * ...

Idealmente estas variaciones buscan mejorar la precisión de la recomendación, pero se valorarán intentos interesantes aunque resulten fallidos en ese aspecto.

Para probar las implementaciones deberá completarse la función `student_test()` para ilustrar la ejecución de las variantes adicionales, y se incluirán las filas que correspondan en la tabla del apartado anterior.

In [ ]:
# Código aquí: clases, funciones...

def student_test():
    # Código de prueba aquí...

### Ejercicio 5 &ndash; Explicación/documentación

(por hacer)